In [ ]:
!pip install fastapi uvicorn ultralytics tensorflow torch requests pyngrok nest-asyncio pypi-json pydantic moviepy pillow  -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 13.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install ffmpeg-python

In [ ]:
import ffmpeg

def trim_video(starting_frame, ending_frame):
  print("In trim video")
  print("Start frame: ", starting_frame)
  print("End frame: ", ending_frame)
  if os.path.exists('/content/video.mp4'):
        os.remove('/content/video.mp4')
        print("Deleted existing 'video.mp4'")
  input_file = ffmpeg.input('/content/origVideo.mp4')
  output_file = ffmpeg.output(input_file.trim(start_frame=starting_frame, end_frame=ending_frame).setpts('PTS-STARTPTS'), 'video.mp4')
  ffmpeg.run(output_file)

In [ ]:
from IPython import display as dp
from ultralytics import YOLO
from IPython.display import display, Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from glob import glob
from random import sample
from PIL import Image, ImageFont, ImageDraw, ImageEnhance, ImageFilter
from pathlib import Path
import os
import shutil
import numpy as np
import cv2
import random
import torch
from tensorflow.keras.preprocessing.image import array_to_img
import requests
from pydantic import BaseModel
import math

class model_input(BaseModel):
  video : str
  x : int
  y : int
  timestamp: int

def get_last_frame(video_url):
  print("In get_last_frame")
  cap = cv2.VideoCapture(video_url)
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

  cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)
  ret, frame = cap.read()
  cap.release()

  cv2.imwrite('/content/last_frame.png', frame)

  return frame

def load_person_dect_model(video_url):
  print("In load_person_dect_model")
  model = YOLO('yolov8m.pt')
  results = model.track(source=video_url, conf=0.25, save=True, tracker='bytetrack.yaml', classes=0)
  return results

def get_tapped_box(results, tap_coord):
    print("In get_tapped_box")
    last_frame_index = len(results) - 1
    last_frame_boxes = results[last_frame_index].boxes.xyxy.tolist()
    required_track_id = None
    min_distance = float('inf')

    for index, box in enumerate(last_frame_boxes):
        x_min, y_min, x_max, y_max = box
        box_center = ((x_min + x_max) / 2, (y_min + y_max) / 2)
        tap_distance = math.sqrt((tap_coord[0] - box_center[0]) ** 2 + (tap_coord[1] - box_center[1]) ** 2)

        if tap_distance < min_distance:
            min_distance = tap_distance
            required_track_id = results[last_frame_index].boxes[index].id.tolist()[0]
            print("Tracking ID: ", required_track_id)

    return required_track_id if required_track_id is not None else None

def get_jersey_color(last_frame_image, tap_coord):
    print("In get_jersey_color")
    cropped_img = None
    player_found = False

    model = YOLO('yolov8m.pt')
    frame_result = model.predict(source='/content/last_frame.png', conf=0.25, save=True, classes=0)
    last_frame_boxes = frame_result[0].boxes.xyxy.tolist()
    print("LAST FRAME BOXES: ")
    print(last_frame_boxes)

    min_distance = float('inf')
    closest_box = None

    print("Tap Coord:", tap_coord)

    for index, box in enumerate(last_frame_boxes):
        x_min, y_min, x_max, y_max = map(int, box)
        box_center = ((x_min + x_max) / 2, (y_min + y_max) / 2)
        tap_distance = math.sqrt((tap_coord[0] - box_center[0]) ** 2 + (tap_coord[1] - box_center[1]) ** 2)

        if tap_distance < min_distance:
            min_distance = tap_distance
            closest_box = box

    print("Min distance: ", min_distance)
    if min_distance > 300:
      return False

    if closest_box is not None:
        x_min, y_min, x_max, y_max = map(int, closest_box)
        y_max = (y_min + y_max) // 2
        cropped_img = last_frame_image[y_min:y_max, x_min:x_max].copy()
        player_found = True
        print("Cropping frame")
        cv2.imwrite('/content/cropped_frame.png', cropped_img)

    if not player_found:
        return False

    hsv_cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2HSV)

    lower_red = np.array([0,50,50])
    upper_red = np.array([10,255,255])
    mask0 = cv2.inRange(hsv_cropped_img, lower_red, upper_red)

    lower_red = np.array([170,50,50])
    upper_red = np.array([180,255,255])
    mask1 = cv2.inRange(hsv_cropped_img, lower_red, upper_red)

    mask_red = mask0+mask1

    cv2.imwrite('/content/mask_red.png', mask_red)

    cropped_img_gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)

    total_pixels = cv2.countNonZero(cropped_img_gray)
    red_pixels = cv2.countNonZero(mask_red)
    red_percentage = (red_pixels / total_pixels) * 100

    red_threshold = 15
    print("Red percentage: ", red_percentage)
    if red_percentage >= red_threshold:
        return True
    else:
        return False

def load_jersey_dect_model():
  print("In load_jersey_dect_model")
  jersey_model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/yolov5ID_640_100epoch.pt')
  jersey_model.load_state_dict(torch.load('/content/drive/MyDrive/yolov5ID_640_100epoch.pt')['model'].state_dict(), strict=False)
  return jersey_model

def clear_output_folder():
  print("In clear_output_folder")
  output_folder = '/content/cropped_images/'

  if os.path.exists(output_folder):
      shutil.rmtree(output_folder)
  os.makedirs(output_folder)

def pre_process_images(results, required_track_id):
  print("In pre_process_images")
  image_paths = []
  output_folder = '/content/cropped_images/'

  for result_index, result in enumerate(results):
      for box_index, box in enumerate(result.boxes):
          if box.id is not None:
            current_track_id = box.id.tolist()[0]

          if current_track_id == required_track_id:
              x_min, y_min, x_max, y_max = box.xyxy.tolist()[0]
              orig_img_bgr = result.orig_img

              orig_img_rgb = cv2.cvtColor(orig_img_bgr, cv2.COLOR_BGR2RGB)

              orig_img = Image.fromarray(orig_img_rgb)

              cropped_img = orig_img.crop((x_min, y_min, x_max, y_max))

              enlarged_size = (cropped_img.width * 2, cropped_img.height * 2)
              cropped_img = cropped_img.resize(enlarged_size, Image.BICUBIC)

              cropped_img_sharpened = cropped_img.filter(ImageFilter.SHARPEN)
              cropped_img_sharpened = cropped_img_sharpened.filter(ImageFilter.SHARPEN)

              image_filename = f'cropped_image_{result_index}_box_{box_index}_sharpened.png'
              image_path = os.path.join(output_folder, image_filename)
              cropped_img_sharpened.save(image_path)

              image_paths.append(image_path)

  return image_paths

def predict_number(img, model):
  print("In predict_number")
  im = array_to_img(img)
  output = model(im)
  results = output.pandas().xyxy[0].to_dict(orient="records")
  for result in results:
      cs = result['class']
      return cs

def get_jersey_number(image_paths, model):
  print("In get_jersey_number")
  num_dict = {}

  for path in image_paths:
    img = cv2.imread(path)
    num = predict_number(img, model)
    if num is not None:
      if num not in num_dict:
        num_dict[num] = 1
      else:
        num_dict[num] += 1

  if bool(num_dict) == False:
    return None

  print(num_dict)

  return max(num_dict, key=num_dict.get)


def download_video(video_url, save_path, starting_frame, ending_frame):
    print("In download_video")
    response = requests.get(video_url, stream=True)

    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024*1024):
                file.write(chunk)
        print("Video downloaded")
        trim_video(starting_frame, ending_frame)
        return True
    else:
        return False



def predict_jersey(video_url, tap_coords, starting_frame, ending_frame):
  dp.clear_output()

  video_filename = 'video.mp4'
  download_path = os.path.join('.', 'origVideo.mp4')
  if not download_video(video_url, download_path, starting_frame, ending_frame):
      return "Failed to download the video"

  last_frame_image = get_last_frame(video_filename)
  if get_jersey_color(last_frame_image, tap_coords):
    results = load_person_dect_model(video_filename)
    bbox_id = get_tapped_box(results, tap_coords)
    if bbox_id == None:
      return "Not a player"
    else:
      model = load_jersey_dect_model()
      clear_output_folder()
      image_paths = pre_process_images(results, bbox_id)
      jersey_number = get_jersey_number(image_paths, model)
      if jersey_number == None:
        return "Cannot identify"
      else:
        return jersey_number
  else:
    return "Not a player (Jersey color is not red)"

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
from pyngrok import ngrok, conf
import nest_asyncio
import json
import math

app = FastAPI()

origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/predict/")
async def predict_jersey_number(input_parameters : model_input):
    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)
    tap_coords = (input_dictionary['x'], input_dictionary['y'])
    timestamp = input_dictionary['timestamp']
    ending_frame = round((timestamp / 1000) * 30)
    print("Ending frame: ", ending_frame)
    starting_frame = ending_frame - 50
    if starting_frame < 0:
      starting_frame = 0
    print("Starting frame: ", starting_frame)
    number = predict_jersey(input_dictionary['video'], tap_coords, starting_frame, ending_frame)
    return {"jersey_number": number}

In [ ]:
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = "2f12n34CoKYGCV5XWBXCgwZ6Frx_4vpCZthbo8DF4eHgkpVoA"

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

In download_video
Video downloaded
In trim video
Start frame:  124
End frame:  174
Deleted existing 'video.mp4'
In get_last_frame
In get_jersey_color

image 1/1 /content/last_frame.png: 384x640 16 persons, 1442.7ms
Speed: 3.9ms preprocess, 1442.7ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict10
LAST FRAME BOXES: 
[[508.8204345703125, 805.0352783203125, 560.9170532226562, 958.8043212890625], [1615.784912109375, 769.68603515625, 1715.174560546875, 916.2235107421875], [1304.4517822265625, 625.646728515625, 1357.087646484375, 738.466796875], [1600.361572265625, 411.3262023925781, 1636.077392578125, 494.4817199707031], [1763.80517578125, 493.08343505859375, 1837.985595703125, 589.64892578125], [1025.79541015625, 539.2002563476562, 1064.63671875, 634.5330200195312], [720.7963256835938, 509.5022277832031, 761.9129638671875, 608.99072265625], [569.3326416015625, 395.3762512207031, 606.2785034179688, 473.3229675292969], [211.5620727539062

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-5-1 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7279825 parameters, 0 gradients, 16.6 GFLOPs
Adding AutoShape... 


In clear_output_folder
In pre_process_images
In get_jersey_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
In predict_number
{14: 2}
INFO:     182.180.38.253:0 - "POST /predict/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [144]


KeyboardInterrupt: 